In [6]:
import numpy as np
import pandas as pd

# 1. OK 데이터 : 휴폐업 이력이 없고 현재까지 영업중인 기업

In [9]:
# Data Loading : 액티브 중소법인 재무보유 -> 기업개요 
active_info = pd.read_csv('./raw_data/active_info.csv')

In [14]:
# Data Loading :  액티브 중소범인 재무보유 -> 기업개요_외감
#기업 외감(External Auditor) : 외감*: 외감(외감법인)은 주식회사 중 자산총액이 120억이 넘는 회사로, 회계법인으로부터 의무적으로 회계감사를 받아야 함.
active_EA = pd.read_csv('./raw_data/active_EA_info.csv')

In [15]:
# Data loading : 액티브 중소법인 재무보유 -> 휴폐업이력
active_history = pd.read_csv('./raw_data/active_history.csv')

In [23]:
# BIZ_NO 기준으로 액티브 중소법인 기업 외감  - 기업 휴폐업 이력 join
df = pd.merge(active_EA, active_history, on = 'BIZ_NO', how  = 'outer')

In [84]:
# OK데이터 = 휴폐업 이력이 없는 기업
# END_DATA가 존재하면 휴폐업 이력이 있는것이므로 해당 기업 drop
OK = df[df['END_DATE'].isna()]

In [28]:
# OK 데이터 인덱스 리셋
OK = OK.reset_index(drop = True)
# OK label : 0
OK['label'] = 0

# 2. NG 데이터 : 폐업한 기업(CLSBZ_GB = 3)

In [33]:
# Data loading : 휴폐업 중소법인 재무보유 -> 기업개요 
closed_info = pd.read_csv('./raw_data/closed_info.csv')

In [86]:
# DATA loading : 휴폐업 중소법인 재무보유 -> 휴폐업이력
closed_history  = pd.read_csv('./raw_data/closed_history.csv')

In [87]:
# 휴폐업 이력 코드 확인
closed_history['CLSBZ_GB'].value_counts()

3    6838
6       1
5       1
Name: CLSBZ_GB, dtype: int64

In [89]:
# 폐업(3) 데이터만 사용
closed_history = closed_history[closed_history['CLSBZ_GB'] == 3].reset_index(drop = True)

In [96]:
# BIZ_NO 기준으로 휴폐업 중소법인 기업개요  - 휴폐업 이력 join
df = pd.merge(closed_info, closed_history, on = 'BIZ_NO', how = 'outer')

In [93]:
# NG데이터 = 폐업한 기업
# 폐업 이력 매칭되지 않은 row 제거
NG = df[~df['EMP_CNT'].isna()].reset_index(drop = True)
# NG데이터 인덱스 리셋
NG = NG.reset_index(drop = True)
# NG label : 1
NG['label'] = 1

# 3. 재무데이터 매칭

In [53]:
# Data loading : 재무데이터.txt
finance = pd.read_csv('./raw_data/finance.txt', sep="\t", encoding = 'cp949')

In [54]:
# OK, NG 데이터와 매칭을 위해 key column명 동일하게 변경
finance = finance.rename(columns = {'사업자번호':'BIZ_NO'})

In [55]:
# 누락값 확인
finance[finance['유동자산'].isna()]

,BIZ_NO,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,...,비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산
280,1018164892,20191231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
281,1018164892,20201231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
282,1018164892,20211231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
336,1018179639,20191231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN,NaN,NaN
337,1018179639,20201231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61737,3018635288,20210930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
61738,3018635288,20211231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
61781,3028107140,20191231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN
61782,3028107140,20201231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000e+12,8.888889e+11,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,NaN,NaN,NaN,NaN


In [56]:
# 누락값 제거
finance = finance[~finance['유동자산'].isna()]

In [99]:
# 주요 결산일 확인 -> 결산 기준일 설정 : 매년 12월 31일
finance['결산년월'].value_counts()

20181231    16560
20191231    14786
20201231    13720
20211231    13681
20180331      291
20190331      256
20200331      222
20210331      211
20220331      198
20180630      196
20190630      186
20200630      154
20210630      147
20190930       82
20180930       81
20200930       67
20210930       63
20220630       29
20180228       20
20180531       18
20190228       17
20210531       17
20190531       16
20220531       16
20200531       15
20200229       13
20210228       12
20180430       12
20191031       12
20181031       12
20190430       11
20180831       11
20211130       11
20191130        9
20220228        9
20201130        9
20210430        9
20200430        8
20201031        8
20190831        8
20220430        8
20181130        8
20211031        8
20210131        8
20200131        7
20220131        7
20200831        6
20210831        6
20180131        5
20190731        4
20190131        4
20210731        4
20180731        3
20200731        3
Name: 결산년월, dtype: int64

In [100]:
# 기업정보와 재무정보를 연도별로 매칭
def match_finance(date):
    f = finance[finance['결산년월'] == date]
    OK_new = pd.merge(OK, f, on = 'BIZ_NO', how = 'outer')
    OK_new = OK_new[(~OK_new['유동자산'].isna())& ~OK_new['CMP_NM1'].isna()].reset_index(drop = True)
    NG_new = pd.merge(NG, f, on = 'BIZ_NO', how = 'outer')
    NG_new = NG_new[(~NG_new['유동자산'].isna())& ~NG_new['CMP_NM'].isna()].reset_index(drop=True)
    return OK_new, NG_new

In [65]:
OK_2018, NG_2018 = match_finance(20181231)

In [66]:
OK_2019, NG_2019 = match_finance(20191231)

In [67]:
OK_2020, NG_2020 = match_finance(20201231)

In [68]:
OK_2021, NG_2021 = match_finance(20211231)

In [69]:
# 재무 결산후 1년 안에 망한 기업으로 필터링 
NG_2018_final = NG_2018[NG_2018['STAT_OCR_DATE'] <= 20191231].reset_index(drop=True)

In [70]:
NG_2019_final = NG_2019[NG_2019['STAT_OCR_DATE'] <= 20201231].reset_index(drop=True)

In [71]:
NG_2020_final = NG_2020[NG_2020['STAT_OCR_DATE'] <= 20211231].reset_index(drop=True)

In [72]:
NG_2021_final = NG_2021[NG_2021['STAT_OCR_DATE'] <= 20221231].reset_index(drop=True)

In [190]:
# 데이터 저장
NG_2021.to_csv('./data/NG_2021.csv', index = False)